In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [95]:
#Squadify!

import json
from datasets import load_dataset

input_filename = '../../data/RadQA'\
                 '/radqa-a-question-answering-dataset-to-improve-comprehension-of-radiology-reports-1.0.0'\
                 '/dev.json'
output_filename = '../../data/RadQA'\
                 '/radqa-a-question-answering-dataset-to-improve-comprehension-of-radiology-reports-1.0.0'\
                 '/dev.jsonl'

with open(input_filename, encoding="utf-8") as f:
    radqa = json.load(f)

with open(output_filename, "w", encoding="utf-8") as f:
    for example in radqa["data"]:
        title = example.get("title", "")
        for paragraph in example["paragraphs"]:
            context = paragraph["context"]  # do not strip leading blank spaces GH-2585
            for qa in paragraph["qas"]:
                question = qa["question"]
                id_ = qa["id"]

                answer_starts = [answer["answer_start"] for answer in qa["answers"]]
                answers = [answer["text"] for answer in qa["answers"]]              
                f.write(
                    json.dumps(
                        {
                    "title": title,
                    "context": context,
                    "question": question,
                    "id": id_,
                    "answers": {
                        "answer_start": answer_starts,
                        "text": answers,
                               },
                        }
                               )
                       )
                f.write("\n")

#ds = load_dataset("json", data_files=output_filename)

In [ ]:
all_contexts = []
all_q = []
for row in s['train']['paragraphs']:
    for d in row:
        all_contexts.append(d["context"])
        for q in d['qas']:
            all_q.append(q["question"])

In [ ]:
import re
c=0
for r in all_contexts:
    #if re.search('findings',r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    #if re.search('final report', r, re.IGNORECASE) and re.search('impression:', r, re.IGNORECASE):
    if re.search('findings and impression', r, re.IGNORECASE):
        print(r)
        print('-'*100)
        

In [1]:
import pickle5 as pickle
with open('../../data/RadQA/ents_spacy_radqa.pkl', 'rb') as f:
    s = pickle.load(f)

In [130]:
import re
c=0
filtered=[]
for ent in s:
    if not re.search(r"\[|\]|\.|(patient['s]*)|baby|report|conclusion|finding*|impression*|procedure", ent, re.IGNORECASE) and len(ent.strip()) >= 3:
        filtered.append(ent.replace('\n',''))
len(filtered)

10352

In [131]:
for i in filtered:
    print(i)

repeat radiograph
problem
Cortical
left calf veins
pulmonary congestion
right lower lung
leukocytosis
dense stool
photon counts
Occipital bone fracture
wheeze
cystic mass
worsening tachypnea
bronchocentric infection
Portable technique
CVVH
axillary surgery
portion
Follow up fracture
heart post
supervening pneumonia
ovarian tumor
Clinical correlation
fact laceration
microscopic hematuria 
duodenal closure
distal superior vena cava
chronic kidney disease
coronary arterial calcification
peak systolic velocity 
filled masses
articular surface
reconstitution
bony
mass effect
Decrease
right lower extremity veins
right neck
small amount
right abdomen
Atrial fibrillation
median sternotomy
distal
Sequences
enhancement/edema
dual-lead cardiac pacemaker
septic source
airways
question pulmonary edema
involutional change
solid hypoechoic mass
tissue samples
status post coronary artery bypass graft surgery
Locally advanced ovarian carcinoma
cardiac variation
hydronephrosis
Humeral head fracture
righ

mastoids are clear
diffuse infiltrates
enterography
pneumothorax/pleural effusions
left hip 
left globe
mediastinum
colon
no definite
THREE
seconds
subperitoneal space
left parapontine stroke
Suboptimal scan
side wall
comparison
focal abnormality
spots
PM  New PA cath
PM  No acute abnormalities
Chest radiograph
Forward LVEF
attenuation structures
central
pulse-wave Doppler images
patellofemoral compartments
Portable AP
weighted images
mild-to-moderate 
Endotracheal reposition
intracerebral arteries
foci
Multiplanar
AM  Status
Accesion
residual intraventricular hemorrhage
Borderline size
interstitial infiltrate
lung field
recommended
venous bleed
contrast per department protocol
liquid
empty
right shin
chronic worsening decubitus ulcer
scanned
trifurcated
carotid portion
tablet
perinephric abnormalities
paranasal sinuses
visceral
right  hilar lymphadenopathy
proximal right humerus
air fluid
patency
hematoma adjacent
left lower lobe consolidation
intercoastal space
twin
cholecystitis
pub

non dilated gallbladder
vertebral artery
pulmonary artery diameter
Followup venous infarcts
G-tube
Right apical opacity
written informed consent
optic nerve
feel
perihilar haziness
pigtail multisidehole
ultrasound were stored
Lower lung volumes 
Burst fracture
volvulus
interim
right posterior communcating artery
cervical soft tissues
prenatally
Duplex
left lower extremity weakness
right temporal bone fracture
Multiplanar reformations
alveolar lung process
extra-hepatic biliary dilatation
clinical exam
present
vertebral body
Small pericardial effusion
granulomatous disease
Chest frontal
right atrium 
uterus 
approximately
peribronchial cuffing
retrocardiac opacities
maxilla
epicardial wire
bilateral L5 pars
dysmorphic features
right frontal lobe
fasting
Enteric contrast
Dense calcification
infrahilar
fixation
mucus plug
restarting tube
basal scars
supine study
status post chest tube
left maxillectomy
right posterior cerebral artery
fracture repair
resolving interstitial edema
small ston

congenitally hypoplastic right
calcium deposits
tumor reoccurred
right calf tenderness
aortic valve, aortic arch
lateral displacement
abnormalitie
major branches
left pelvis
vertebral flows
risks
Right scapular fracture
pleural thickening
blood
joint effusion
active 
tiny bilateral
moderate-sized left pleural effusion
lying
DSsd
Consolidation 
chest wall trauma
cavities
Chest x ray
MEDICATIONS
urinary bladder look
left hilum
analysis
Ventricles
acute pathology
centimeter
right hepatic vein under fluoroscopic guidance
DAH
intraventricular
interstitial lines
atherosclerotic calcification
abdominal wall
cecal volvulus
obstruction
subtraction images
Heart probably normal
OG tube position
brachiocephalic artery
traversing right S1 nerve root
Preop
visualized heart
C6 level 
Structure
bowel 
lateral tibial plateau
4-chamber
right calf veins
Segments
ultrasound guidance
intraabdominal air
originating from
Mural calcification
Post-reconstruction
thrombi
focal confluent
stable
Dilated loops
int

mid trachea
large bowel
frontal bone
cervical airway
dense mass
Soft tissue density right
Follow up
side
subareolar mass
atelectatic change
fluoroscopic image
interval volumes
S/P AVR
INDICATION
small-bowel injury
tip terminating
enteral tube
left kidneys
New placement right
right complex
axial plane
Questioned right upper pleural
poorly
abnormal enhancement
Pain
left-sided disc herniation
extraluminal air
pigtail catheter overlying
Right PICC line
extra-axial hemorrhage
atelectasis
area around pancreas
air pocket
small right pleural effusion persists
normal course
activity
left transverse sinus
MV repair
dialysis catheter
inferior
cervical soft tissue
interval
chest radiographs
time
MDCT-acquired contiguous images
characteristic distribution
Dobbhoff tube terminating
flare
Indeterminate fracture
VIEW
flex/ext scan
hepatitis C cirrhosis
generalized demineralization
gastrojejunostomy tube
hemotympanum
right knee
left breast mass look
treated
body of stomach
posterior endplate osteophyte

No pneumothorax
Minimal
acute territorial infarction
No IV contrast
Axial MDCT images
accessory splenic tissue
Delayed enhancement
Type I dens fracture
DCT
right internal capsule
microperforation
popliteal veins
backing
Left small nonobstructing kidney stone
anterior ethmoid cells
Alcohol abuse
congestive heart failure
sphenoid 
T2-
C4-5 and C5-6
generalized anasarca
ingestion
Clear lungs
STIR-weighted images
Doppler waveform evaluation
post infectious
linear calcifications
right heart border
lateral fracture plate
acalculous cholecystitis
evaluating
Air fluid
Fatty liver
midline catheter
left plueral effusion pocket
asymmetrical degenerative
interval progression
episode of arrest
changes
bilateral ground-glass opacification
Brachial veins
Type II fracture
Small pleural effusions
vascular catheter
ethmoid air cells
hemodynamic monitoring
pulmonary contusion
Mediastinal contour
acromioclavicular joint degenerative change
anuerysm
aortic pseudoaneurysm
faint
bilateral pleural effusions
N

In [70]:
w = 'groin hematoma.'
w.replace('.','')

'groin hematoma'